In [2]:
import numpy as np
from scipy.linalg import block_diag
from sympy import Matrix

### Generate Combination Matrix

In [57]:
def generate_combination_matrices():
    """
    Output to modeldata.py
    ----------------------
    Data.inbound_combination_matrices: dict
        Dictionary of block diagonal matrices containing the production
        efficiency of a factory {product: associated matrix}. Each
        product corresponds to a matrix with:
            - Columns: ∑|F| (total number of factories across all products)
            - Rows: # Factories (total number of factories)

    Data.outbound_combination_matrices: dict
        Dictionary of block diagonal matrices to apply outbound constraints
        on a per-factory basis. {product: associated matrix}. Each product
        corresponds to a matrix with:
            - Column: ∑|FxC| (total number of factories x customers
            across all products)
            - Rows: # Factories (total
            number of factories)
    """

    # Verify inputs type
    assert isinstance(
        Data.efficiency_per_product,
        dict), 'Efficiency per product must be given in a dictionary'
    assert isinstance(Data.factory_names,
                      dict), 'Factory names must be given in a dictionary'

    # Verify inputs values
    assert len(Data.factory_list
               ) > 0, 'Number of factories to optimize must be positive'
    assert np.all(np.hstack(list(Data.efficiency_per_product.values())) > 0
                  ), 'Efficiency must be positive'
    assert np.all(
        np.array([len(prod) for prod in Data.factory_names.values()]) <= len(
            Data.factory_list)), 'There are more factory names than allowed'

    # Build the combination matrix
    """
    Starting from the inside out, we first iterate over all the product. 
    For a given product "p", we put the corresponding efficiency value  
    in the place of the factories that produce the product and [] in the 
    place of factories that don't. Then, we form each product into a block 
    using the block_diag function. After which, we build the matrix using 
    block_diag on all the previous blocks. Lastly, add a negative sign per
    the model.
    """

    # Generator to iterate over efficiency elements
    efficiency_array = (
        eff_arr
        for eff_arr in np.hstack(list(Data.efficiency_per_product.values())))

    Data.inbound_combination_matrix = -block_diag(*[
        block_diag(*block) for block in [[
            next(efficiency_array)
            if factory in Data.factory_names[product] else []
            for factory in Data.factory_list
        ] for product in Data.product_list]
    ])
    """
    Starting from the inside out, we first iterate over all the product. 
    For a given product "p", we put a list [1 * #customer buying "p"] 
    in the place of the factories that product and [] in the place of 
    factories that don't. Then, we form each product into a block using
    the block_diag function. After which, we build the matrix using 
    block_diag on all the previous blocks.
    """

    Data.outbound_combination_matrix = block_diag(*[
        block_diag(*block)
        for block in [
            [
                [1] * Data.customer_sizes[product]
                if factory in Data.factory_names[product] else []
                for factory in Data.factory_list]
            for product in Data.product_list
        ]
   ])

    # Verify matrix dimension
    # Inbound combination matrix dimension == (Σ|C|, Σ#Fx#P)
    assert Data.inbound_combination_matrix.shape == (
        len(Data.factory_list) * len(Data.product_list), Data.dimF
    ), 'Dimension of inbound combination matrix is incorrect (Σ|C|, Σ#Fx#P)'

    # Outbound combination matrix dimension == (Σ|FxC|, Σ#Fx#P)
    assert Data.outbound_combination_matrix.shape == (
        len(Data.factory_list) * len(Data.product_list),
        Data.dimFC), 'Dimension of combination matrix is incorrect (Σ|FxC|, ' \
                     'Σ#Fx#P)'

    # Split the matrix into dictionary
    # Inbound combination dictionary
    Data.inbound_combination_matrices = dict(
        zip(
            Data.product_list,
            np.split(Data.inbound_combination_matrix,
                     len(Data.product_list),
                     axis=0)))

    # Outbound combination dictionary
    Data.outbound_combination_matrices = dict(
        zip(
            Data.product_list,
            np.split(Data.outbound_combination_matrix,
                     len(Data.product_list),
                     axis=0)))

### Unit Test

In [36]:
class Data:
    pass

In [47]:
class DataTest:
    """Class for generating random test inputs and verify that the combination
    matrix is constructed correctly"""

    no_products = None
    customer_names = None
    no_factories = None
    no_customers = None

    @classmethod
    def generate_random_inputs(cls):
        """Generate random Data inputs"""

        cls.no_customers = np.random.randint(1, 20)
        cls.no_factories = np.random.randint(1, 10)
        cls.no_products = np.random.randint(1, 10)

        if cls.no_customers == 1:
            cls.customer_names = dict(
                zip(range(cls.no_products), [np.array([0])] * cls.no_products))
        else:  # Allow for one customer
            cls.customer_names = {
                prod: np.sort(
                    np.random.choice(np.arange(cls.no_customers),
                                     size=np.random.randint(
                                         1, cls.no_customers),
                                     replace=False))
                for prod in range(cls.no_products)
            }

        if cls.no_factories == 1:
            Data.factory_names = dict(
                zip(range(cls.no_products), [np.array([0])] * cls.no_products))
        else:  # Allow for one factory
            Data.factory_names = {
                prod: np.sort(
                    np.random.choice(np.arange(cls.no_factories),
                                     size=np.random.randint(
                                         1, cls.no_factories),
                                     replace=False))
                for prod in range(cls.no_products)
            }

        # Inputs from Data
        Data.factory_list = list(range(cls.no_factories))
        Data.product_list = list(range(cls.no_products))

        Data.customer_sizes = {
            prod: len(cls.customer_names[prod])
            for prod in range(cls.no_products)
        }

        Data.factory_sizes = {
            prod: len(Data.factory_names[prod])
            for prod in range(cls.no_products)
        }

        Data.efficiency_per_product = dict(
            zip(range(cls.no_products), [
                np.random.uniform(0.8, 1, Data.factory_sizes[prod])
                for prod in range(cls.no_products)
            ]))

        Data.dimF = sum(Data.factory_sizes.values())
        Data.dimC = sum(Data.customer_sizes.values())

        Data.dimFC = sum([
            Data.factory_sizes[prod] * Data.customer_sizes[prod]
            for prod in range(cls.no_products)
        ])

    @classmethod
    def alternative_combination_matrix(cls):
        """Construct the combination matrix in an alternative method"""
        
        # Alternative Inbound Combination Matrix, 
        # Here we start by choosing out the appropriate index and 
        # replace those elements with the efficiency values
        DataTest.inbound_combination_matrix = np.zeros(
            (cls.no_factories * cls.no_products,
             cls.no_factories * cls.no_products))

        matrix_index = np.hstack([
            cls.no_factories * prod + Data.factory_names[prod]
            for prod in Data.product_list
        ])

        DataTest.inbound_combination_matrix[matrix_index, matrix_index] = np.hstack(
            list(Data.efficiency_per_product.values()))

        DataTest.inbound_combination_matrix = -DataTest.inbound_combination_matrix[:, ~np.all(
            DataTest.inbound_combination_matrix == 0, axis=0)]

        # Alternative Outbound Combination Matrix
        # Here we just iterate over all the products
        # and find out which factories produce and which 
        # don't. Then we either put [1, ..., 1] or [] as
        # appropriate 

        block_list = []

        for prod in Data.product_list:

            subblock_list = []

            for fac in Data.factory_list:

                if fac in Data.factory_names[prod]:
                    subblock_list.append([1] * Data.customer_sizes[prod])
                else:
                    subblock_list.append([])

            block_list.append(block_diag(*subblock_list))

        DataTest.outbound_combination_matrix = block_diag(*block_list)

In [48]:
# Generate Random Inputs
DataTest.generate_random_inputs()

# Generate the combination matrix
generate_combination_matrix()

# Generate the alternative combination matrix
DataTest.alternative_combination_matrix()

In [56]:
# Check inbound equality
np.array_equal(Data.inbound_combination_matrix, DataTest.inbound_combination_matrix)

# Check outbound equality
np.array_equal(Data.outbound_combination_matrix, DataTest.outbound_combination_matrix)

True